# Test lstm1 model for phishing URL detection

## Imports

In [1]:
import numpy as np
import os
import pathlib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential

import tensorflow_text as tf_text

## Params

In [2]:
dataset_dir = pathlib.Path("../URL_dataset/")
model_dir = pathlib.Path('../URL_models/')
checkpoint_dir = pathlib.Path('../URL_checkpoints/')

In [5]:
model_name = "lstm1"
batch_size = 16
rest_split_ratio = 0.2
test_split_ratio = 0.2

unicode_max = 1114111

## Load dataset

In [6]:
train_ds = keras.preprocessing.text_dataset_from_directory(
  dataset_dir,
  validation_split=rest_split_ratio,
  subset="training",
  label_mode = 'binary',
  seed=132,
  batch_size=batch_size)

Found 45343 files belonging to 2 classes.
Using 36275 files for training.


In [7]:
rest_ds = keras.preprocessing.text_dataset_from_directory(
  dataset_dir,
  validation_split=rest_split_ratio,
  subset="validation",
  label_mode='binary',
  seed=132,
  batch_size=batch_size)

Found 45343 files belonging to 2 classes.
Using 9068 files for validation.


In [8]:
rest_batches = rest_ds.cardinality().numpy()
print(rest_batches)

567


In [9]:
val_ds = rest_ds.skip(int(rest_batches * test_split_ratio // batch_size))
test_ds = rest_ds.take(int(rest_batches * test_split_ratio // batch_size))

In [10]:
class_names = train_ds.class_names

In [11]:
class_names

['benign', 'phishing']

## Explore dataset

In [12]:
b = test_ds.take(1)

In [13]:
for batch in b:
    test_urls = batch[0]
    test_labels = batch[1]

In [14]:
for url, label in zip(test_urls.numpy(), test_labels.numpy()):
    print(url, class_names[int(label[0])])

b'http://perezhilton.com/2015-05-12-prince-harry-baby-fever-adorable-new-zealand-visit/?from=featured' benign
b'http://distractify.com/post/related/id/553568574a0c4b4443bec65b/skip/20/limit/10/back/0' benign
b'http://www.tsekourasp.gr/language/en-GB/msg/ln.php' phishing
b'http://hollywoodlife.com/2015/05/02/ian-somerhalder-engagement-nikki-reed-khloe-kardashian-lamar-odom-wedding-pics/' benign
b'http://www.sz-leteng.com/images/?http://us.battle.net/login/en/?ref=http://bopgkotus.battle.net/d3/en/index&amp;app=com-d3' phishing
b'http://askubuntu.com/questions/106028/is-it-possible-to-run-64-bit-mongo-db-on-my-32-bit-pae-enabled-ubuntu' benign
b'http://serverfault.com/questions/528627/nginx-returning-404-with-python-urllib-but-fine-if-accessed-from-browser' benign
b'http://twitter.com/home?status=%E3%83%8C%E3%81%91%E3%82%8B%EF%BC%81%E3%80%90%E4%BA%BA%E5%A6%BB%E3%83%BB%E7%86%9F%E5%A5%B3%E3%80%91+http%3A%2F%2Fero-video.net%2Ft%2FRVIEW8Nj0X3Idi8o+%E5%B7%A8%E4%B9%B3%E4%BA%BA%E5%A6%BB%E3%81%A

## Vectorize

Cannot use the vectorization layers since it is not supported in TFJS; we can rather have a custom tokenizer which can be then exported as json then loaded in JS

In [15]:
tokenizer = tf_text.UnicodeCharTokenizer()

In [16]:
tokenized_urls = tokenizer.tokenize(test_urls)

## Load model

In [17]:
# change filename to the model you want to load
model_filename = "2021-08-19_20;05;32.036830_lstm1_valacc0.9948_e10_b16.tf"
model = keras.models.load_model(
    model_dir / model_filename,
    compile=True)

## Test the model

In [48]:
tu

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([104., 116., 116., 112.,  58.,  47.,  47., 119., 119., 119.,  46.,
       116., 115., 101., 107., 111., 117., 114.,  97., 115., 112.,  46.,
       103., 114.,  47., 108.,  97., 110., 103., 117.,  97., 103., 101.,
        47., 101., 110.,  45.,  71.,  66.,  47., 109., 115., 103.,  47.,
       108., 110.,  46., 112., 104., 112.], dtype=float32)>

In [46]:
tu=tf.cast(tokenized_urls[2], tf.float32)

In [47]:
tf.sigmoid(model.predict(tf.expand_dims(tu, 0))).numpy()

array([[0.99999464]], dtype=float32)

In [49]:
url = "http://example.com"

In [50]:
tokenizer.tokenize(url)

<tf.Tensor: shape=(18,), dtype=int32, numpy=
array([104, 116, 116, 112,  58,  47,  47, 101, 120,  97, 109, 112, 108,
       101,  46,  99, 111, 109])>

In [51]:
tf.sigmoid(model.predict(tf.expand_dims(tokenizer.tokenize(url), 0))).numpy()

array([[0.40307313]], dtype=float32)

In [ ]:
i = 2

In [29]:
p = tf.sigmoid(model.predict(tf.expand_dims(tokenized_urls[i], 0))).numpy()

In [32]:
tf.expand_dims(tokenized_urls[i], 0)

AttributeError: 
        'EagerTensor' object has no attribute 'astype'.
        If you are looking for numpy-related methods, please run the following:
        import tensorflow.python.ops.numpy_ops.np_config
        np_config.enable_numpy_behavior()

In [31]:
p

array([[0.9999989]], dtype=float32)

In [23]:
pred = tf.round(tf.sigmoid(model.predict(tokenized_urls))).numpy().astype(int)

In [30]:
predictions = [class_names[p[0]] for p in pred]

In [33]:
for i, (url, p, label) in enumerate(zip(test_urls.numpy(), predictions, test_labels.numpy())):
    print(url, "Prediction (model):", p, "Truth:", class_names[int(label[0])])
    print(tokenized_urls[i])

b'http://perezhilton.com/2015-05-12-prince-harry-baby-fever-adorable-new-zealand-visit/?from=featured' Prediction (model): benign Truth: benign
tf.Tensor(
[104 116 116 112  58  47  47 112 101 114 101 122 104 105 108 116 111 110
  46  99 111 109  47  50  48  49  53  45  48  53  45  49  50  45 112 114
 105 110  99 101  45 104  97 114 114 121  45  98  97  98 121  45 102 101
 118 101 114  45  97 100 111 114  97  98 108 101  45 110 101 119  45 122
 101  97 108  97 110 100  45 118 105 115 105 116  47  63 102 114 111 109
  61 102 101  97 116 117 114 101 100], shape=(99,), dtype=int32)
b'http://distractify.com/post/related/id/553568574a0c4b4443bec65b/skip/20/limit/10/back/0' Prediction (model): benign Truth: benign
tf.Tensor(
[104 116 116 112  58  47  47 100 105 115 116 114  97  99 116 105 102 121
  46  99 111 109  47 112 111 115 116  47 114 101 108  97 116 101 100  47
 105 100  47  53  53  51  53  54  56  53  55  52  97  48  99  52  98  52
  52  52  51  98 101  99  54  53  98  47 115 107 105 

## Evaluate (test)

In [18]:
test_results = model.evaluate(tokenized_urls, test_labels, batch_size=16)
print("test loss, test acc:", test_results)

1/1 [==============================] - 1s 953ms/step - loss: 3.3692e-04 - accuracy: 1.0000
test loss, test acc: [0.00033692337456159294, 1.0]
